https://www.kaggle.com/c/competitive-data-science-final-project

In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
%matplotlib inline 

import utility.utility as ut
from preprocessing.createSets import *
from preprocessing.Sets import Sets
from utility.ML import ML

########### switches ################################

saveName='Oct18-2018_1'
saveFolder = saveName
saveplots=False
createSubmit = False
if not os.path.exists(saveFolder) and (saveplots or createSubmit): 
    print 'creating folder:',os.getcwd()+'/'+saveFolder
    os.mkdir(saveFolder)
if(saveplots): print "SAVING PLOTS!"
if(createSubmit): print "WILL CREATE SUBMIT FILE!"

############# Options/Args #########################

verbose=False
lag_length = 2
diff = False
diffRel = False 
addPriceRange = True

target = 'shop_item_cnt_month'
# target = 'shop_item_cnt_month_diff(0-1)'

#these columns will be dropped in createTrain/Val/Test --> Depracated
col_targets=[]

####### mean Encode (with Reg) #####

meanEncode=True #this is just necessary condition for mean encoding. but need to turn on individual switches below to include columsn of target encoding.
meanEncodeCol=[ #this is for lag features.
#              'shop',
#              'item',
#              'item_cat',
            ]

agg_targ = {'item_cnt_day':'sum'} #target_encoding!
# agg_targ = {'item_cnt_day':'mean'} #target_encoding!

Regularize = True
enc_cnt_per_shop = False
enc_cnt_per_item = False
enc_cnt_per_item_cat = False
enc_priceRange = False

targEnc_to_Reg = {}
NaN_targEnc = {}

if(enc_cnt_per_shop):targEnc_to_Reg.update({'shop_id':'shop'})
if(enc_cnt_per_item):targEnc_to_Reg.update({'item_id':'item'})
if(enc_cnt_per_item_cat):targEnc_to_Reg.update({'item_category_id':'item_cat'})
if(enc_priceRange):targEnc_to_Reg.update({'price_range':'price_range'})

if(enc_cnt_per_shop):NaN_targEnc.update({'shop':-999})
if(enc_cnt_per_item):NaN_targEnc.update({'item':-999})
if(enc_cnt_per_item_cat):NaN_targEnc.update({'item_cat':-999})
if(enc_priceRange):NaN_targEnc.update({'price_range':-999})

####################################

#columns to keep
col_to_keep = [
                'shop_id',
                'item_id',
                #'item_price',
                'item_category_id',
                'item_cnt_day',
                ]

groupby_list = ['shop_id','item_id']

agg_dict = {
            #'item_price':'mean',
            'item_category_id':'mean',
            'item_cnt_day':'sum',
            }

    
opt = {
        'verbose':verbose,
        'lag_length':lag_length,
        'diff':diff,
        'diffRel':diffRel,
        'target':target,
        'col_to_keep':col_to_keep,
        'groupby_list':groupby_list,
        'agg_dict':agg_dict,
        'agg_targ':agg_targ,
        'col_targets':col_targets,
        'meanEncode':meanEncode,
        'meanEncodeCol':meanEncodeCol,
        'targEnc_to_Reg':targEnc_to_Reg,
        'NaN_targEnc':NaN_targEnc,
        } 


#clipTarget = True
lowerClip = 0
upperClip = 20
if target=='shop_item_cnt_month_diff(0-1)':
    lowerClip = -20
    upperClip = 20    

########################################################

#del Sets
Sets = Sets(**opt)

dup_ids = Sets.checkDuplicates()
Sets.convertDatetime()
Sets.addItemCategoryId()
Sets.addYMcolumn()
bin_edges=[0,10,100,200,500,1000,2500,5000,20000,999999]
Sets.binPrice(bin_edges)
Sets.splitDataByYear()
data = Sets.getData()


lag_length: 2
diff: False
diffRel: False
target: shop_item_cnt_month

target encoding: {'item_cnt_day': 'sum'}



In [2]:
data['sales_train'].head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,year,month,Y_M,price_range
1972631,2014-09-01,20,54,0,58.0,1.0,40,2014,9,2014_9,"(10, 100]"
2069405,2014-10-20,21,55,1,4490.0,1.0,76,2014,10,2014_10,"(2500, 5000]"
1552547,2014-04-04,15,55,1,4490.0,1.0,76,2014,4,2014_4,"(2500, 5000]"
1815466,2014-07-06,18,55,1,4490.0,1.0,76,2014,7,2014_7,"(2500, 5000]"
1906676,2014-08-04,19,55,1,4490.0,1.0,76,2014,8,2014_8,"(2500, 5000]"


In [3]:
df_temp = data['sales_train'][data['sales_train']['year']==2013][['date_block_num','year','month']].sort_values(by='date_block_num').drop_duplicates().set_index('date_block_num')
df_temp.transpose()

date_block_num,0,1,2,3,4,5,6,7,8,9,10,11
year,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013
month,1,2,3,4,5,6,7,8,9,10,11,12


In [4]:
df_temp = data['sales_train'][data['sales_train']['year']==2014][['date_block_num','year','month']].sort_values(by='date_block_num').drop_duplicates().set_index('date_block_num')
df_temp.transpose()

date_block_num,12,13,14,15,16,17,18,19,20,21,22,23
year,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014,2014
month,1,2,3,4,5,6,7,8,9,10,11,12


In [5]:
df_temp = data['sales_train'][data['sales_train']['year']==2015][['date_block_num','year','month']].sort_values(by='date_block_num').drop_duplicates().set_index('date_block_num')
df_temp.transpose()

date_block_num,24,25,26,27,28,29,30,31,32,33
year,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015
month,1,2,3,4,5,6,7,8,9,10


# Adding isItemNew feature

In [13]:
df_now,_ = Sets.createDateblockSet([22])
df_prev,_ = Sets.createDateblockSet([21])

adding isItemNew feature ...
adding isItemNew feature ...


In [14]:
print df_now.shape
df_now.head()

(45755, 11)


,shop_id,item_id,item_category_id,shop_item_cnt_month,shop_item_id,shop_cnt_month,item_cnt_month,item_cat_cnt_month,isItemNew,shop_item_cnt_month_lag_1,shop_item_cnt_month_lag_2
0,2,486,73,2.0,2_486,1192.0,170.0,402.0,False,1.0,1.0
1,2,491,73,1.0,2_491,1192.0,17.0,402.0,False,1.0,0.0
2,2,806,49,1.0,2_806,1192.0,54.0,760.0,False,1.0,14.0
3,2,949,23,1.0,2_949,1192.0,31.0,5275.0,False,0.0,0.0
4,2,950,19,1.0,2_950,1192.0,17.0,6134.0,False,0.0,0.0


In [15]:
print df_prev.shape
df_prev.head()

(42595, 11)


,shop_id,item_id,item_category_id,shop_item_cnt_month,shop_item_id,shop_cnt_month,item_cnt_month,item_cat_cnt_month,isItemNew,shop_item_cnt_month_lag_1,shop_item_cnt_month_lag_2
0,2,32,40,2.0,2_32,945.0,37.0,13639.0,False,2.0,0.0
1,2,464,49,1.0,2_464,945.0,32.0,903.0,False,0.0,0.0
2,2,482,73,1.0,2_482,945.0,73.0,443.0,False,2.0,0.0
3,2,486,73,1.0,2_486,945.0,183.0,443.0,False,1.0,0.0
4,2,491,73,1.0,2_491,945.0,18.0,443.0,False,0.0,0.0


In [16]:
int_list = list( set(df_now['item_id'].values).intersection(set(df_prev['item_id']))) 
print 'num of intersection',len(int_list)
only_in_now = list(set(df_now['item_id'].values) -  set(int_list))
print 'num only in now',len(only_in_now)

num of intersection 4958
num only in now 1364


In [17]:
df_prev[df_prev['item_id']==1534]

,shop_id,item_id,item_category_id,shop_item_cnt_month,shop_item_id,shop_cnt_month,item_cnt_month,item_cat_cnt_month,isItemNew,shop_item_cnt_month_lag_1,shop_item_cnt_month_lag_2


In [18]:
df_now['IsItemNew_test'] = df_now['item_id'].isin(only_in_now)

In [19]:
df_now['IsItemNew_test'] = ~df_now['item_id'].isin(df_prev['item_id'].values)

In [20]:
df_now['IsItemNew_test'] = df_now['item_id'].apply(lambda x: False if x in df_prev['item_id'].values else True)

In [21]:
df_now[df_now['item_id']==1534][['item_id','IsItemNew_test']]

,item_id,IsItemNew_test
10,1534,True
1634,1534,True
4926,1534,True
7118,1534,True
9212,1534,True
10198,1534,True
11239,1534,True
11892,1534,True
12797,1534,True
15080,1534,True


# Create train/val/test set

In [22]:
# split by date block num
x_train, y_train = Sets.createDateblockSet([22])
x_val, y_val = Sets.createDateblockSet([22])
x_test = Sets.createTestSet()
print x_train.shape
print x_val.shape

(45755, 11)
(45755, 11)


In [12]:
len(x_train[x_train['isItemNew']==1]['item_id'].unique())

1364